# **Progetto di Matteo Calvanico e Filippo Monti**

---
---

## Per quanto riguarda i CSV...

Per questo progetto useremo due CSV, eccoli nel dettaglio:
#### Fonti

> * [CSV 1](https://www.kaggle.com/datasets/mahmoudshogaa/us-crime-rates-1960-2014/)
> * [CSV 2](https://www.kaggle.com/datasets/guillemservera/us-unemployment-rates)

#### Contenuto

> * Il CSV 1 contiene il numero di totale di crimini, divisi per categoria, avvenuti negli U.S.A dal 1960 al 2014.
>
> * Il CSV 2 contiene le percentuali dei cittadini senza lavoro negli U.S.A divise in diversi fattori (Es: età), a partire dal 1940 fino al 2023. La percentuale è stata registrta il 1° di ogni mese.

---

## Scopo

L'obiettivo finale che ci siamo posti è quello di capire se c'è una correlazione fra i dati riportati dai due csv.
In particolare vogliamo dimostrare o confutare che:

> All'aumentare del tasso di disoccupazione aumentino anche i crimini, concentrandoci nei furti di vario tipo.

&nbsp;

Successivamente tramite i dati che abbiamo proveremo a fare una previsione se i crimini e la disoccupazione aumenteranno o diminuiranno nei prossimi anni.

---

# !!! Da cancellare prima della consegna !!!

## Idee implementative

* Dentro le slides di Statistics, c'è una parte intitolata: 'Measures of relationship between variables' (linear correlation coefficient = indice di correlazione di Person) e per verificare se i dati ottenuti si siano varificati per puro caso c'è: 'Hypothesis testing'.

* Anche la rigressione lineare è utile

* Sicuramente per il secondo obiettivo bisogna vedere le slides 'Time Series'

* Alcuni esempi per la correlazione fra due df sono negli esercizi di Statistica, nella cartella houses

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import int64 #Ci servirà nella modifica del csv

#### Carichiamo i due CSV e li mostriamo

In [77]:
df_crime = pd.read_csv("CSV1.csv")
df_employment = pd.read_csv("CSV2.csv")

In [78]:
df_crime.head()

,Year,Population,Total,Violent,Property,Murder,Forcible_Rape,Robbery,Aggravated_assault,Burglary,Larceny_Theft,Vehicle_Theft
0,1960,179323175,3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200
1,1961,182992000,3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000
2,1962,185771000,3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800
3,1963,188483000,4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300
4,1964,191141000,4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800


In [79]:
df_employment

,date,overall_rate,age_16_17_rate,age_16_19_rate,age_18_19_rate,age_16_24_rate,age_20_24_rate,age_25_34_rate,age_25_54_rate,age_35_44_rate,age_45_54_rate,age_20plus_rate,age_25plus_rate,age_55plus_rate
0,1948-01-01,3.4,9.4,8.5,8.3,6.8,5.8,3.0,2.5,2.0,2.2,3.0,2.6,2.9
1,1948-02-01,3.8,13.1,10.0,8.2,7.7,6.3,3.3,2.9,2.5,2.8,3.3,2.9,3.1
2,1948-03-01,4.0,13.0,10.5,8.9,8.7,7.6,2.9,2.8,2.7,2.6,3.5,2.8,3.0
3,1948-04-01,3.9,11.2,9.5,8.6,7.8,6.8,3.4,3.0,2.9,2.6,3.5,3.0,3.0
4,1948-05-01,3.5,6.5,7.0,7.9,6.7,6.6,3.0,2.7,2.6,2.4,3.3,2.8,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,2023-08-01,3.8,11.9,12.3,12.7,8.6,7.1,3.9,3.1,3.0,2.5,3.4,3.0,2.7
908,2023-09-01,3.8,12.8,11.8,11.1,8.4,7.0,4.0,3.2,2.8,2.8,3.5,3.0,2.7
909,2023-10-01,3.8,14.7,13.1,12.5,8.8,7.0,4.2,3.2,2.9,2.4,3.5,3.1,2.7
910,2023-11-01,3.7,11.4,11.4,11.5,8.0,6.6,3.9,3.1,2.9,2.6,3.4,3.1,2.9


#### Modifica CSV 

Nel secondo csv le varie percentuali sono divise per diversi fattori e suddivise per mesi di ogni anno. Come prima operazione cancelliamo tutte le percentuali tranne quella totale, unica davvero utile al nostro obiettivo.

Successivamente andremo a raggruppare le percentuali per anno e non per mese

In [80]:
#Cancelliamo le colonne futili al nostro obiettivo
df_employment = df_employment.drop(columns=['age_16_17_rate', 'age_16_19_rate', 'age_18_19_rate', 'age_16_24_rate', 'age_20_24_rate', 'age_25_34_rate', 'age_25_54_rate', 'age_35_44_rate', 'age_45_54_rate', 'age_20plus_rate', 'age_25plus_rate', 'age_55plus_rate'])


#Convertiamo la data da String a DateTime per semplicità
df_employment['date'] = pd.to_datetime(df_employment['date'])


#Ora possiamo prendere l'anno con facilità
df_employment['anno'] = df_employment['date'].dt.year


#Raggruppiamo per anno e facciamo una media di tutte le percentuali di quell'anno
df_employment = df_employment.groupby('anno', as_index=False)['overall_rate'].mean()


#Arrotondiamo a due cifre dopo la virgola
df_employment['overall_rate'] = df_employment['overall_rate'].round(2)


#Rinomiamo le colonne
df_employment.columns = ['Anno', 'Percentuale_Disoccupati']


#Mostriamo il nuovo Data Frame
df_employment

,Anno,Percentuale_Disoccupati
0,1948,3.75
1,1949,6.05
2,1950,5.21
3,1951,3.28
4,1952,3.02
...,...,...
71,2019,3.68
72,2020,8.11
73,2021,5.35
74,2022,3.63
